In [23]:
#Import Libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

import warnings
warnings.filterwarnings('ignore')

#https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

### Load Data

In [35]:
# load dataset

df = pd.read_csv('data/diagnosis.csv')
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})
df


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,NaN
565,926682,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,NaN
566,926954,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,NaN
567,927241,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,NaN


In [36]:
# split into input (X) and output (Y) variables
X = df.iloc[:,2:18]
Y = df["diagnosis"]

## Define NN Model

In [42]:
# Function to create model, required for KerasClassifier
def create_model():

    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(16,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [43]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0) 

## Grid Search

In [46]:
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100, 150, 200]
param_grid = dict(batch_size=batch_size, epochs=epochs)

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3) #3 fold cross validation
grid_result = grid.fit(X, Y)

In [47]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.899907 using {'batch_size': 10, 'epochs': 150}
0.815446 (0.060683) with: {'batch_size': 10, 'epochs': 10}
0.868300 (0.064914) with: {'batch_size': 10, 'epochs': 50}
0.884071 (0.055774) with: {'batch_size': 10, 'epochs': 100}
0.899907 (0.040887) with: {'batch_size': 10, 'epochs': 150}
0.845187 (0.064947) with: {'batch_size': 10, 'epochs': 200}
0.519530 (0.277524) with: {'batch_size': 20, 'epochs': 10}
0.880498 (0.021598) with: {'batch_size': 20, 'epochs': 50}
0.885761 (0.032266) with: {'batch_size': 20, 'epochs': 100}
0.875225 (0.047321) with: {'batch_size': 20, 'epochs': 150}
0.898116 (0.026119) with: {'batch_size': 20, 'epochs': 200}
0.727810 (0.095013) with: {'batch_size': 40, 'epochs': 10}
0.801504 (0.061501) with: {'batch_size': 40, 'epochs': 50}
0.875225 (0.034448) with: {'batch_size': 40, 'epochs': 100}
0.877054 (0.041592) with: {'batch_size': 40, 'epochs': 150}
0.859417 (0.028544) with: {'batch_size': 40, 'epochs': 200}
0.726047 (0.171004) with: {'batch_size': 60, 'epoch